# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

#data science - standard library
import pandas as pd
import numpy as np

#requests
import requests
from pprint import pprint
import time

#regression
from scipy.stats import linregress
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "NT_cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [7]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

602

In [8]:
cities_sub = cities[0:10]
cities_sub

['lagoa',
 'carndonagh',
 'nikolskoye',
 'ushuaia',
 'xifeng',
 'rikitea',
 'hilo',
 'gubkinskiy',
 'port alfred',
 'saint-philippe']

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [11]:
#init lists to hold parsed data
lats = []
longs = []
temps = []
feel_likes = []
humids = []
pressures = []
speeds = []
cloudiness = []
cities_found = []

for i in range(len(cities)):
    #get the city
    city = cities[i]
    
    #create the url
    units = "imperial"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units={units}&appid={weather_api_key}"
#     print(url)

    #make the request
    response = requests.get(url)
#     print(response.status_code)
    
    #error check
    if response.status_code == 200:
        data = response.json()
#         pprint(data)
        
        try:
            #extract the data
            lat = data["coord"]["lat"]
            long = data["coord"]["lon"]
            temp = data["main"]["temp"]
            feels_like = data["main"]["feels_like"]
            humidity = data["main"]["humidity"]
            pressure = data["main"]["pressure"]
            speed = data["wind"]["speed"]
            clouds = data["clouds"]["all"]
            city_name = data["name"]

            #save the data
            lats.append(lat)
            longs.append(long)
            temps.append(temp)
            feel_likes.append(feels_like)
            humids.append(humidity)
            pressures.append(pressure)
            speeds.append(speed)
            cloudiness.append(clouds)
            cities_found.append(city_name)
        except Exception as e:
            print(f"Through exception for city {city}: {e}")
    
    elif response.status_code == 404:
        print(f"Missing data in OpenWeatherAPI for {city}")
    else:
        print(response.status_code)
        print("Oh! The API is broken.")
        
    #print every 5
    if (i % 5 == 0):
        print(f"Got data for city index: {i} of {len(cities)}")
    
    #sleep for requests
    time.sleep(1)
        
# make the dataframe
df = pd.DataFrame()
df["City"] = cities_found
df["Latitude"] = lats
df["Longitude"] = longs
df["Temperature"] = temps
df["Feels Like"] = feel_likes
df["Humidity"] = humids
df["Pressure"] = pressures
df["Wind Speed"] = speeds
df["Cloudiness"] = speeds

df.head(10)

Got data for city index: 0 of 602
Got data for city index: 5 of 602
Got data for city index: 10 of 602
Missing data in OpenWeatherAPI for grand river south east
Got data for city index: 15 of 602
Got data for city index: 20 of 602
Missing data in OpenWeatherAPI for taolanaro
Missing data in OpenWeatherAPI for sentyabrskiy
Got data for city index: 25 of 602
Got data for city index: 30 of 602
Missing data in OpenWeatherAPI for belushya guba
Got data for city index: 35 of 602
Got data for city index: 40 of 602
Missing data in OpenWeatherAPI for nizhneyansk
Got data for city index: 45 of 602
Missing data in OpenWeatherAPI for tsihombe
Got data for city index: 50 of 602
Missing data in OpenWeatherAPI for tostamaa
Got data for city index: 55 of 602
Missing data in OpenWeatherAPI for illoqqortoormiut
Got data for city index: 60 of 602
Missing data in OpenWeatherAPI for attawapiskat
Got data for city index: 65 of 602
Got data for city index: 70 of 602
Got data for city index: 75 of 602
Got dat

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Lagoa,39.05,-27.98,60.89,52.72,73,1022,15.73,15.73
1,Carndonagh,55.25,-7.27,48.20,42.04,88,1014,8.77,8.77
2,Nikolskoye,59.70,30.79,45.07,35.20,81,1011,13.42,13.42
3,Ushuaia,-54.80,-68.30,55.40,42.04,47,994,18.34,18.34
4,Xifeng,42.74,124.72,34.29,21.25,69,1026,15.23,15.23
5,Rikitea,-23.12,-134.97,71.82,66.15,73,1016,17.74,17.74
6,Hilo,19.73,-155.09,71.47,73.45,88,1015,8.05,8.05
7,Gubkinskiy,64.43,76.50,33.49,24.26,98,988,10.31,10.31
8,Port Alfred,-33.59,26.89,65.07,62.80,75,1012,7.90,7.90
9,Saint-Philippe,-21.36,55.77,74.71,69.12,65,1021,17.22,17.22


In [16]:
df.to_csv(output_data_file, index=False)

# Continue in NT_WeatherPy_Visualization
